In [ ]:
from functions import *
import json

# Creating a P2SH multisig transaction

In this section we'll create a P2SH transaction from scratch in python. We'll go through each part of the transaction, how it's constructed, signed, and we'll test it using bitcoin core in regtest mode.

## Prerequisite knowledge
- For all notebooks:
    - A high level understanding of the bitcoin. e.g. [Mastering Bitcoin](https://github.com/bitcoinbook/bitcoinbook) by Andreas Antonopoulos UTXO model, in particular [Chapter 6](https://github.com/bitcoinbook/bitcoinbook/blob/develop/ch06.asciidoc).
    - A conceptual understanding of [hash functions](https://www.thesslstore.com/blog/what-is-a-hash-function-in-cryptography-a-beginners-guide).
    - [Hexadecimal notation](https://inst.eecs.berkeley.edu/~cs61bl/r//cur/bits/decimal-binary-hex.html?topic=lab28.topic&step=2&course=) and [endianness](https://www.freecodecamp.org/news/what-is-endianness-big-endian-vs-little-endian/).


- Specific to this notebook:
    - SHA256, HASH256, HASH160 - '[Hash Functions chapter](./0.2-hash-functions.ipynb)'
    - Base58 addresses - '[Addresses chapter](./0.4-addresses.ipynb)'
    - Bitcoin Script basics - '[Bitcoin Script chapter](./0.3-bitcoin-script.ipynb)'
    - TestShell setup - '[P2PKH chapter](./1.2-p2pkh.ipynb)'

### Create a P2SH multisig UTXO

In order to create a transaction spending from a P2SH multisig UTXO, we'll first need to create the UTXO that is locked with a multisig script. For this example we'll use a 2 of 3 multisig script. We'll create a three pubkeys and manually conscruct the multisig script. Then we'll fund it using the bitcoind wallet created in the setup step.

#### Create a multisig redeemScript 
For more on this step, review the 'Bitcoin Script' notebook.

In [ ]:
privkey1 = bytes.fromhex("1111111111111111111111111111111111111111111111111111111111111111")
pubkey1 = privkey_to_pubkey(privkey1)

privkey2 = bytes.fromhex("2222222222222222222222222222222222222222222222222222222222222222")
pubkey2 = privkey_to_pubkey(privkey2)

privkey3 = bytes.fromhex("3333333333333333333333333333333333333333333333333333333333333333")
pubkey3 = privkey_to_pubkey(privkey3)

# "21" is the length of a 33 byte (compressed) pubkey in hex notation
redeemScript = bytes.fromhex(
    "52" 
    + "21"
    + pubkey1.hex()
    + "21"
    + pubkey2.hex()
    + "21"
    + pubkey3.hex()
    + "53"
    + "ae")

print(redeemScript.hex())

#### Convert the redeemScript to a P2SH address

For more on this step, review the 'Addresses' notebook.

In [ ]:
address_to_spend = script_to_p2sh(redeemScript, "regtest")
print(address_to_spend)

#### Fund the 'sender' with 2.001 btc (0.001 btc is for the next tx fee)

Now that we have the address, we'll fund it using the `TestShell` commands. For more on these steps you can view the [function definitions](./functions/setup_testshell.py) or look at the first [P2PKH notebook](./1.2-p2pkh.ipynb) example.

In [ ]:
node = setup_testshell()
txid_to_spend, index_to_spend = fund_address(node, address_to_spend, 2.001)
print(f"txid: {txid_to_spend}, {index_to_spend}")

## Spending a P2PH (multisig) UTXO

Now that we have some funds locked up in a P2SH multisig utxo, we can create a transaction spending from it. Let's say we want to send 1.5 btc to the address `mkxwE7XtVYJKepoD2hbHnDjftuMQ1k6deE`.

From our previous example we know this corresponds to a scriptPubkey of `76a9143bc28d6d92d9073fb5e3adf481795eaf446bceed88ac`, so we will skip the decoding step. For more on decoding base58 addresses, refer to the 'Addresses' chapter.

In [ ]:
receiver_spk = bytes.fromhex("76a9143bc28d6d92d9073fb5e3adf481795eaf446bceed88ac")

### Create an unsigned P2SH transaction

The first thing we'll do is define the inputs and outputs of our transaction.

In [ ]:
# Note we have already defined a few variables we need to create our transaction:
# The input utxo txid and index: `txid_to_spend` and `index_to_spend`
# The input private key and public key: `privkey1`, `pubkey1`, ...

# Set our outputs
# Create a new pubkey to use as a change output.
change_privkey = bytes.fromhex("4444444444444444444444444444444444444444444444444444444444444444")
change_pubkey = privkey_to_pubkey(change_privkey)

# Determine our output scriptPubkeys and amounts (in satoshis)
output1_value_sat = int(float("1.5") * 100000000)
output1_spk = receiver_spk
output2_value_sat = int(float("0.5") * 100000000)
output2_spk = bytes.fromhex("76a914") + hash160(change_pubkey) + bytes.fromhex("88ac")

Now that we've defined everything we need, we can fill in the fields we need to create our unsigned transaction. What makes a transaction 'unsigned' is that the input's scriptSig, the field where the signature goes, is empty. This first step is necessary as the signature will cover the whole transaction (using SIGHASH_ALL). In a later chapter we will cover other sighash types and how they are signed.

In [ ]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
txid = (bytes.fromhex(txid_to_spend))[::-1]
index = index_to_spend.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# use 0xffffffff unless you are using OP_CHECKSEQUENCEVERIFY, locktime, or rbf
sequence = bytes.fromhex("ffff ffff")

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

# OUTPUTS
# 0x02 for out two outputs
output_count = bytes.fromhex("02")

# OUTPUT 1 
output1_value = output1_value_sat.to_bytes(8, byteorder="little", signed=True)
# 'output1_spk' already defined at the start of the script

# OUTPUT 2
output2_value = output2_value_sat.to_bytes(8, byteorder="little", signed=True)
# 'output2_spk' already defined at the start of the script

outputs = (
    output1_value
    + pushbytes(output1_spk)
    + output2_value
    + pushbytes(output2_spk)
)

# LOCKTIME
locktime = bytes.fromhex("0000 0000")

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)
print("unsigned_tx: ", unsigned_tx.hex())

We can decode this raw transaction to inspect it and see that it has all the information we need apart from the scriptSig.

In [ ]:
decoded = node.decoderawtransaction(unsigned_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

Before we can sign this transaction there is one final step we need to do. We need to replace the empty scriptSig with the scriptPubkey of the input we are signing over. If we had multiple inputs, we would need to do this step for each input. We will cover signing transactions with multiple inputs in a later chapter.

Since we are spending from a multisig redeemScript, we'll need to create two signatures, then put them in the scriptSig. Before signing the transaction we need to **replace the empty scriptSig with the redeemScript**. This is similar to the P2PKH example, except we use the redeemScript rather than the input's scriptPubkey. 

In [ ]:
# replace the empty scriptSig with the input scriptPubkey
inputs = (
    txid
    + index
    + varint_len(redeemScript)
    + redeemScript
    + sequence
)

# tx hex to sign
tx_to_sign = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)

Now we are ready to hash this transaction and produce two ecdsa signatures on it. 

Before hashing the transaction with hash256, we append the sighash flag. In this example we'll use the most commonly used SIGHASH_ALL flag, meaning the signature guarantees the input will only be used in a transaction with these exact inputs and outputs.

Note that when we append the sighash flag to the transaction, we use 4 bytes, however when we append the sighash flag to the end of the signature itself we only use 1 byte.

In [ ]:
# Append the sighash flag to the transaction
sighash_flag = bytes.fromhex("0100 0000") # SIGHASH_ALL
sighash_preimage = tx_to_sign + sighash_flag

# Create sigHash to be signed
sighash = hash256(sighash_preimage)

# Sign the sigHash with the input privkey1
signing_key1 = ecdsa.SigningKey.from_string(privkey1, curve=ecdsa.SECP256k1) 
signature1 = signing_key1.sign_digest(sighash, sigencode=ecdsa.util.sigencode_der_canonize)

# Append SIGHASH_ALL to the signature
signature1 = signature1 + bytes.fromhex("01")

# Sign the sigHash with the input privkey2
signing_key2 = ecdsa.SigningKey.from_string(privkey2, curve=ecdsa.SECP256k1) 
signature2 = signing_key2.sign_digest(sighash, sigencode=ecdsa.util.sigencode_der_canonize)

# Append SIGHASH_ALL to the signature
signature2 = signature2 + bytes.fromhex("01")

# Combine the signatures into the final scriptSig
sig_script_signed = (
    bytes.fromhex("00") # Add an extra "00" for the CheckMultiSig bug
    + pushbytes(signature1)
    + pushbytes(signature2)
    + pushbytes(redeemScript)
)

# tx_in with our new sigScript containing the signatures we just created
inputs_signed = (
    txid
    + index
    + varint_len(sig_script_signed)
    + sig_script_signed
    + sequence
)

# the final signed transaction
signed_tx = (
    version
    + input_count
    + inputs_signed
    + output_count
    + outputs
    + locktime
)

print("signed transaction: ",signed_tx.hex())

### Broadcast the transaction (on regtest mode)
If we get back a txid (32 byte hash), then it means the tx was successfully broadcast! If we just want to see if the transaction would have been accepted, but without broadcasting it, we can use the `testmempoolaccept` command (commented out).

In [ ]:
new_tx_txid = node.sendrawtransaction(signed_tx.hex())
print(new_tx_txid)

In [ ]:
decoded = node.decoderawtransaction(signed_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

## Quiz
- 1. Given a P2SH address, is it possible to know what type of redeem script was used to create it? Why not?
- 2. How does the cost compare for sending bitcoins to a 2-of-3 multisig or a 2-of-2 multisig? What about the cost of spending from them?
- 3. Does the order of the signatures matter when spending from a multisig? If so, how is the order of the signatures determined?
- 4. What are some privacy concerns when using P2SH outputs?

## Answers
- 1. No. The address is an encoding of the HASH160 of the redeem script.
- 2. Sending to them is the same as the P2SH scriptPubKey will be the same length (since it commits to the hash of the script). Spending from the 2-of-3 script will cost more than the 2-of-2 multisig as the redeemScript needs to be included in the scriptSig, and it is longer for the 2-of-3 script.
- 3. Yes. The order of the signatures must be in the same order as they appear in the redeemScript.
- 4. When the P2SH outputs are spent, the scripts get revealed on chain. This reveals the other ways the bitcoin could have been spent. The script may be combined with other on-chain data to uncover other informations such as which outputs are change outputs.

## Exercises

1. To verify for yourself that the order of the signatures does matter, try rearranging signature 1 and signature 2. What error message do you encounter when trying to broadcast the transaction?